In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
feature_x_sample_processed = pd.read_table(
    PATH_DICT["feature_x_sample.processed.tsv"], index_col=0
)

In [ ]:
feature_x_sample_processed_01 = ccal.normalize_df(
    feature_x_sample_processed, 1, "0-1", raise_for_bad=False
)

In [ ]:
if SELECT_FEATURE_AUTOMATICALLY:

    value_name = "{} Raw Signal Standard Deviation".format(FEATURE_ALIAS)

    features = ccal.select_series_low_and_high_index(
        feature_x_sample_processed_01.std(axis=1),
        title="{} Rank".format(value_name),
        value_name=value_name,
    )[1]

else:

    features = feature_x_sample_processed_01.index

print("Selected {} {}.".format(features.size, FEATURE_ALIAS))

In [ ]:
if SELECT_SAMPLE_AUTOMATICALLY:

    value_name = "{} Raw Signal Standard Deviation".format(SAMPLE_ALIAS)

    samples = ccal.select_series_low_and_high_index(
        feature_x_sample_processed_01.std(),
        title="{} Rank".format(value_name),
        value_name=value_name,
    )[1]

else:

    samples = feature_x_sample_processed_01.columns

print("Selected {} {}.".format(samples.size, SAMPLE_ALIAS))

In [ ]:
raw_signal = feature_x_sample_processed_01.loc[features, samples]

In [ ]:
raw_signal.to_csv(PATH_DICT["feature_x_sample.raw_signal.tsv"], sep="\t")

ccal.summarize_feature_x_sample(
    raw_signal,
    feature_x_sample_alias="Raw Signal",
    feature_alias=FEATURE_ALIAS,
    sample_alias=SAMPLE_ALIAS,
    feature_x_sample_value_name="Signal",
    plot_max_size=PLOT_MAX_SIZE,
)

raw_signal

In [ ]:
feature_context = pd.read_table(
    PATH_DICT["feature_x_sample.feature_context.tsv"], index_col=0
)

feature_context_negative_signal = -feature_context.clip(upper=0)

feature_context_positive_signal = feature_context.clip(lower=0)

In [ ]:
sample_context = pd.read_table(
    PATH_DICT["feature_x_sample.sample_context.tsv"], index_col=0
).T

sample_context_negative_signal = -sample_context.clip(upper=0)

sample_context_positive_signal = sample_context.clip(lower=0)

In [ ]:
feature_context_negative_signal = ccal.normalize_df(
    feature_context_negative_signal, 1, "0-1", raise_for_bad=False
)

feature_context_positive_signal = ccal.normalize_df(
    feature_context_positive_signal, 1, "0-1", raise_for_bad=False
)

In [ ]:
sample_context_negative_signal = ccal.normalize_df(
    sample_context_negative_signal, 1, "0-1", raise_for_bad=False
)

sample_context_positive_signal = ccal.normalize_df(
    sample_context_positive_signal, 1, "0-1", raise_for_bad=False
)

In [ ]:
if "feature" in ELEMENT_TYPES and "sample" in ELEMENT_TYPES:

    context_negative_signal = (
        feature_context_negative_signal * sample_context_negative_signal
    )

    context_positive_signal = (
        feature_context_positive_signal * sample_context_positive_signal
    )

elif "feature" in ELEMENT_TYPES:

    context_negative_signal = feature_context_negative_signal

    context_positive_signal = feature_context_positive_signal

elif "sample" in ELEMENT_TYPES:

    context_negative_signal = sample_context_negative_signal

    context_positive_signal = sample_context_positive_signal

In [ ]:
ccal.summarize_feature_x_sample(
    context_negative_signal,
    feature_x_sample_alias="Context Negative Signal",
    feature_alias=FEATURE_ALIAS,
    sample_alias=SAMPLE_ALIAS,
    feature_x_sample_value_name="Signal",
    plot_max_size=PLOT_MAX_SIZE,
)

context_negative_signal

In [ ]:
ccal.summarize_feature_x_sample(
    context_positive_signal,
    feature_x_sample_alias="Context Positive Signal",
    feature_alias=FEATURE_ALIAS,
    sample_alias=SAMPLE_ALIAS,
    feature_x_sample_value_name="Signal",
    plot_max_size=PLOT_MAX_SIZE,
)

context_negative_signal

In [ ]:
if SELECT_FEATURE_AUTOMATICALLY:

    value_name = "{} Context Negative Signal Sum".format(FEATURE_ALIAS)

    context_negative_features = ccal.select_series_low_and_high_index(
        context_negative_signal.sum(axis=1),
        title="{} Rank".format(value_name),
        value_name=value_name,
    )[1]

    value_name = "{} Context Positive Signal Sum".format(FEATURE_ALIAS)

    context_positive_features = ccal.select_series_low_and_high_index(
        context_positive_signal.sum(axis=1),
        title="{} Rank".format(value_name),
        value_name=value_name,
    )[1]

else:

    context_negative_features = context_negative_signal.index

    context_positive_features = context_positive_signal.index

print(
    "Selected {} context negative and {} context positive {}.".format(
        context_negative_features.size, context_positive_features.size, FEATURE_ALIAS
    )
)

In [ ]:
if SELECT_SAMPLE_AUTOMATICALLY:

    value_name = "{} Context Negative Signal Sum".format(SAMPLE_ALIAS)

    context_negative_samples = ccal.select_series_low_and_high_index(
        context_negative_signal.sum(),
        title="{} Rank".format(value_name),
        value_name=value_name,
    )[1]

    value_name = "{} Context Positive Signal Sum".format(SAMPLE_ALIAS)

    context_positive_samples = ccal.select_series_low_and_high_index(
        context_positive_signal.sum(),
        title="{} Rank".format(value_name),
        value_name=value_name,
    )[1]

else:

    context_negative_samples = context_negative_signal.columns

    context_positive_samples = context_positive_signal.columns

print(
    "Selected {} context negative and {} context positive {}.".format(
        context_negative_samples.size, context_positive_samples.size, SAMPLE_ALIAS
    )
)

In [ ]:
context_negative_signal_selected = context_negative_signal.loc[
    context_negative_features, context_negative_samples
]

In [ ]:
context_positive_signal_selected = context_positive_signal.loc[
    context_positive_features, context_positive_samples
]

In [ ]:
if "negative" in CONTEXTS and "positive" in CONTEXTS:

    context_negative_signal_selected.index = (
        "(-) {}".format(feature_name)
        for feature_name in context_negative_signal_selected.index
    )

    context_positive_signal_selected.index = (
        "(+) {}".format(feature_name)
        for feature_name in context_positive_signal_selected.index
    )

    context_signal = pd.concat(
        (context_negative_signal_selected, context_positive_signal_selected)
    )

elif "negative" in CONTEXTS:

    context_signal = context_negative_signal_selected

elif "positive" in CONTEXTS:

    context_signal = context_positive_signal_selected

In [ ]:
context_signal.to_csv(PATH_DICT["feature_x_sample.context_signal.tsv"], sep="\t")

ccal.summarize_feature_x_sample(
    context_signal,
    feature_x_sample_alias="Context Signal",
    feature_alias=FEATURE_ALIAS,
    sample_alias=SAMPLE_ALIAS,
    feature_x_sample_value_name="Signal",
    plot_max_size=PLOT_MAX_SIZE,
)

context_signal